In [40]:
import duckdb

output_data_folder = '/data/experiments'

In [41]:
con = duckdb.connect()
con.install_extension("spatial")
con.load_extension("spatial")

# 1.0 Total private dwellings and private dwellings per square kilometer for Ottawa
These values are from the 2021 Census of Population

In [42]:
con.execute("""
DROP TABLE IF EXISTS geo_data;
CREATE TABLE geo_data AS
SELECT
    geo.da_dguid,
    cop.count_total_4,
    CAST(
        ROUND(
            (cop.count_total_1 / (ST_AREA_SPHEROID(geo.geom) / 1000000.0)), 0
        ) AS INTEGER
    ) AS count_total_4_per_square_km,
    geo.geom
FROM
    'https://data.dataforcanada.org/processed/statistics_canada/census_of_population/2021/tabular/da_2021.parquet' AS cop,
    'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/da_2021.parquet' AS geo
WHERE geo.csd_name IN ('Ottawa') AND cop.da_dguid = geo.da_dguid;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## 1.1 Export result as a GeoJSON

In [43]:
con.execute(f"""
COPY geo_data TO '{output_data_folder}/da_2021_private_dwellings.geojson'
WITH (
  FORMAT GDAL,
  DRIVER 'GeoJSON',
  GEOMETRY_TYPE 'POLYGON',
  SRS 'EPSG:4326'
);
""")

## 1.2 Export result as file geodatabase

In [38]:
con.execute(f"""
COPY geo_data TO '{output_data_folder}/da_2021_private_dwellings.gdb'
WITH (
  FORMAT GDAL,
  DRIVER 'OpenFileGDB',
  GEOMETRY_TYPE 'POLYGON',
  SRS 'EPSG:4326'
);
""")

## 1.3 Export result as GeoParquet

In [39]:
con.execute(f"""
COPY geo_data TO '{output_data_folder}/da_2021_private_dwellings.parquet' (FORMAT PARQUET);
""")